In [1]:
from tensorflow.keras.utils import to_categorical
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import numpy as np
%matplotlib inline
seed = 0
np.random.seed(seed)
import tensorflow as tf
tf.random.set_seed(seed)
import os


import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
from callbacks import all_callbacks

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l1

from qkeras.qlayers import QDense, QActivation
from qkeras.quantizers import quantized_bits, quantized_relu
import tensorflow.compat.v1 as tf1

2023-07-19 01:49:28.227540: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-19 01:49:28.323293: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


#### start processing dataset

In [2]:
df = pd.read_csv('./breast-cancer-wisconsin.csv', sep = ';')
print (np.shape(df))

(683, 11)


In [3]:
X = df.drop('Y', axis = 1).values
y = df.Y
print(y)

0      2
1      2
2      2
3      2
4      2
      ..
678    2
679    2
680    3
681    3
682    3
Name: Y, Length: 683, dtype: int64


In [10]:
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(sparse=False)
Y_reshaped = [[label] for label in y]
Y_encoded = encoder.fit_transform(Y_reshaped)

/home/edge/miniconda3/envs/hls4ml-tutorial/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [11]:
Y_encoded

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       ...,
       [0., 1.],
       [0., 1.],
       [0., 1.]])

In [12]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
X_train,X_test,Y_train,Y_test = train_test_split(X,Y_encoded,test_size=0.3,random_state=42)

In [13]:
sc = MinMaxScaler(feature_range=(0,0.9))
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [14]:
print("Shape of X_train: ",X_train.shape)
print("Shape of X_test: ", X_test.shape)
print("Shape of y_train: ",Y_train.shape)
print("Shape of y_test",Y_test.shape)

Shape of X_train:  (478, 10)
Shape of X_test:  (205, 10)
Shape of y_train:  (478, 2)
Shape of y_test (205, 2)


In [ ]:
# from sklearn.neural_network import MLPClassifier
# from sklearn.datasets import make_classification
# from sklearn.model_selection import train_test_split

# clf = MLPClassifier(beta_1=0.004044058262057914, beta_2=0.2692099545241596,
#               epsilon=0.4100816459563625, hidden_layer_sizes=3, max_iter=150,
#               momentum=0.8221177331942455, nesterovs_momentum=False,
#               solver='lbfgs', validation_fraction=0.511318982546456,alpha=0.0001).fit(X_train, Y_train)
# clf.score(X_test, Y_test)

In [ ]:
# dump(clf,'./Seeds_argykokk_clf.joblib')

### store the weights, biases of the loaded model

In [ ]:
from tensorflow.keras.utils import to_categorical
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import numpy as np

import tensorflow as tf
import os
import pandas as pd
import seaborn as sb

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l1
from callbacks import all_callbacks
from qkeras.qlayers import QDense, QActivation
from qkeras.quantizers import quantized_bits, quantized_relu,quantized_po2
import tensorflow.compat.v1 as tf1
from tensorflow_model_optimization.python.core.sparsity.keras import prune, pruning_callbacks, pruning_schedule
from tensorflow_model_optimization.sparsity.keras import strip_pruning
from qkeras.utils import model_save_quantized_weights
from sklearn.preprocessing import MinMaxScaler

In [32]:
# model=Sequential()

# model.add(QDense(5, input_shape=(10,), name='fc1', kernel_quantizer=quantized_bits(8,0,alpha=1,use_stochastic_rounding=True),bias_quantizer=quantized_bits(8,0,alpha=1),
#                 kernel_initializer='lecun_uniform', kernel_regularizer=l1(0.0001)   ))
# model.add(QActivation(activation=quantized_relu(8,0,use_stochastic_rounding=False), name='relu1'))
# model.add(QDense(3, name='fc2',
#                 kernel_quantizer=quantized_bits(8,0,alpha=2,use_stochastic_rounding=True), bias_quantizer=quantized_bits(8,0,alpha=1),
#                 kernel_initializer='lecun_uniform', kernel_regularizer=l1(0.0001 ) ))
# model.add(QActivation(activation=quantized_relu(8,0,use_stochastic_rounding=False), name='relu2'))
# model.add(QDense(2, name='output',
#                 kernel_quantizer=quantized_bits(8,0,alpha=2,use_stochastic_rounding=True), bias_quantizer=quantized_bits(8,0,alpha=1),
#                 kernel_initializer='lecun_uniform', kernel_regularizer=l1(0.0001 ) ))
# model.add(Activation(activation='softmax', name='softmax'))

In [33]:
# adam = Adam(lr=0.02)

# model.compile(optimizer=adam, loss=['categorical_crossentropy'], metrics=['accuracy'])
# callbacks= all_callbacks( outputDir = 'BC_classification_prune')

# model.fit(X_train, Y_train, batch_size=1,
#           epochs=5,validation_split=0.2, verbose=1, shuffle=True,
#           callbacks = callbacks.callbacks);
# model.compile(optimizer=adam, loss=['categorical_crossentropy'], metrics=['accuracy'])
# model_save_quantized_weights(model, "test_weights")

Epoch 1/5
  1/382 [..............................] - ETA: 5:30 - loss: 0.1995 - accuracy: 1.0000WARNING:tensorflow:Callback method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0031s vs `on_train_batch_end` time: 0.0033s). Check your callbacks.


365/382 [===========================>..] - ETA: 0s - loss: 0.5588 - accuracy: 0.8164
***callbacks***
saving losses to BC_classification_prune/losses.log

Epoch 1: val_loss improved from inf to 0.40112, saving model to BC_classification_prune/KERAS_check_best_model.h5

Epoch 1: val_loss improved from inf to 0.40112, saving model to BC_classification_prune/KERAS_check_best_model_weights.h5

Epoch 1: saving model to BC_classification_prune/KERAS_check_model_last.h5

Epoch 1: saving model to BC_classification_prune/KERAS_check_model_last_weights.h5

***callbacks end***

382/382 [==============================] - 3s 5ms/step - loss: 0.5509 - accuracy: 0.8194 - val_loss: 0.4011 - val_accuracy: 0.9375 - lr: 0.0010
Epoch 2/5
366/382 [===========================>..] - ETA: 0s - loss: 0.2805 - accuracy: 0.9454
***callbacks***
saving losses to BC_classification_prune/losses.log

Epoch 2: val_loss improved from 0.40112 to 0.24927, saving model to BC_classification_prune/KERAS_check_best_model.h5



NameError: name 'model_save_quantized_weights' is not defined

In [38]:
model = tf.keras.models.load_model("Breast_Cancer_model")
accuracy=model.evaluate(X_test,Y_test)

7/7 [==============================] - 0s 3ms/step - loss: 0.1004 - accuracy: 0.9902


In [39]:
layer_1=10
layer_2=5
layer_3=3
layer_4=2
int_bits=0
sign_bit=1
bits=8

In [42]:
import estimate as es

In [43]:
from importlib import reload
reload(es)

<module 'estimate' from '/home/edge/Desktop/argykokk/hls4ml-tutorial/networks/breast_cancer/estimate.py'>

In [55]:
param=1
reuse=100

In [56]:
%%time
#input_num, neurons_num, layer_id, model
luts, ffs = es.estimate(10,5,0,model,reuse,param)
luts, ffs = es.estimate(5,3,2,model,reuse,param)
luts, ffs = es.estimate(3,2,4,model,reuse,param)

zero weight are:  0
Mul ins = 26 and Max muls = 1 and Saved muls = 25 Reuse factor = 100
Muxes LUTS: 325
LUT cost1=: 62 cost2= 371 bias acc= 70 mult acc630
LUTs prediction: 1133
Initial muls: 50 Real muls: 50 Initial neurons: 5 Tuned neurons: 5
FFs prediction: 1672
zero weight are:  0
Mul ins = 7 and Max muls = 1 and Saved muls = 6 Reuse factor = 100
Muxes LUTS: 78
LUT cost1=: 62 cost2= 110 bias acc= 42 mult acc168
LUTs prediction: 382
Initial muls: 15 Real muls: 15 Initial neurons: 3 Tuned neurons: 3
FFs prediction: 903
zero weight are:  0
Mul ins = 1 and Max muls = 1 and Saved muls = 0 Reuse factor = 100
Muxes LUTS: 0
LUT cost1=: 62 cost2= 56 bias acc= 28 mult acc56
LUTs prediction: 202
Initial muls: 6 Real muls: 6 Initial neurons: 2 Tuned neurons: 2
FFs prediction: 734
CPU times: user 21.4 ms, sys: 3.46 ms, total: 24.9 ms
Wall time: 24.3 ms


/home/edge/miniconda3/envs/hls4ml-tutorial/lib/python3.10/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator LinearRegression from version 1.1.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/edge/miniconda3/envs/hls4ml-tutorial/lib/python3.10/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator LinearRegression from version 1.1.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/edge/miniconda3/envs/hls4ml-tutorial/lib/python3.10/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator LinearRegression from version 1.1.2 when 

In [41]:
import hls4ml
import plotting

config = hls4ml.utils.config_from_keras_model(model, granularity='name')
config['LayerName']['softmax']['exp_table_t'] = 'ap_fixed<18,8>'
config['LayerName']['softmax']['inv_table_t'] = 'ap_fixed<18,4>'
print("-----------------------------------")
plotting.print_dict(config)
print("-----------------------------------")
hls_model = hls4ml.converters.convert_from_keras_model(
    model, hls_config=config, output_dir='Breast_cancer/hls4ml_prj', part='xc7z007s-clg225-2'
)
hls_model.compile()

y_qkeras = model.predict(np.ascontiguousarray(X_test))
y_hls = hls_model.predict(np.ascontiguousarray(X_test))

/home/edge/miniconda3/envs/hls4ml-tutorial/lib/python3.10/site-packages/hls4ml/converters/__init__.py:24: UserWarning: WARNING: Pytorch converter is not enabled!
  warnings.warn("WARNING: Pytorch converter is not enabled!", stacklevel=1)


None
Interpreting Sequential
Topology:
Layer name: fc1_input, layer type: InputLayer, input shapes: [[None, 10]], output shape: [None, 10]
Layer name: fc1, layer type: QDense, input shapes: [[None, 10]], output shape: [None, 5]
Layer name: relu1, layer type: Activation, input shapes: [[None, 5]], output shape: [None, 5]
Layer name: fc2, layer type: QDense, input shapes: [[None, 5]], output shape: [None, 3]
Layer name: relu2, layer type: Activation, input shapes: [[None, 3]], output shape: [None, 3]
Layer name: output, layer type: QDense, input shapes: [[None, 3]], output shape: [None, 2]
Layer name: softmax, layer type: Softmax, input shapes: [[None, 2]], output shape: [None, 2]
-----------------------------------
Model
  Precision:         fixed<16,6>
  ReuseFactor:       1
  Strategy:          Latency
  BramFactor:        1000000000
  TraceOutput:       False
LayerName
  fc1_input
    Trace:           False
    Precision
      result:        fixed<16,6>
  fc1
    Trace:           Fal